# Catastro

In [2]:
# listado de ubicacion shape de Lotes de bogota con geometria de punto.
[a.path for a in dbutils.fs.ls("/FileStore/tables/shp/Lotes_centroide/")]

['dbfs:/FileStore/tables/shp/Lotes_centroide/Puntos_lote.cpg',
 'dbfs:/FileStore/tables/shp/Lotes_centroide/Puntos_lote.dbf',
 'dbfs:/FileStore/tables/shp/Lotes_centroide/Puntos_lote.prj',
 'dbfs:/FileStore/tables/shp/Lotes_centroide/Puntos_lote.shp',
 'dbfs:/FileStore/tables/shp/Lotes_centroide/Puntos_lote.shx']

In [ ]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
from sedona.spark import *
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

from sedona.register.geo_registrator import SedonaRegistrator
# configuración de spark y Sedona para datos geograficos
spark = SparkSession.\
    builder.\
    master("local[*]").\
    appName("Sedona App").\
    config("spark.serializer", KryoSerializer.getName).\
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName) .\
    config("spark.kryoserializer.buffer.max", "200gb").\
    getOrCreate()

#SedonaRegistrator.registerAll()
SedonaContext.create(spark)

In [ ]:
#import geopandas as gpd
#lote = gpd.read_file("/dbfs/FileStore/tables/shp/Lotes_centroide/Puntos_lote.shp")

# LOTES FILTRADOS PLMB
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.utils.adapter import Adapter
sc = spark.sparkContext
sc.setSystemProperty("sedona.global.charset", "utf8")
#Lotes
Lotes = ShapefileReader.readToGeometryRDD(sc, "dbfs:/FileStore/tables/shp/Lotes_centroide/")
Isocrona = ShapefileReader.readToGeometryRDD(sc, "dbfs:/FileStore/tables/shp/isocronas/")
Lotes_df = Adapter.toDf(Lotes, spark)
Isocrona_df = Adapter.toDf(Isocrona, spark)

In [ ]:
Lotes.CRSTransform('epsg:4326','epsg:4326')
Isocrona.CRSTransform('epsg:4326','epsg:4326')

In [ ]:
Isocrona.rawSpatialRDD.filter(lambda x: [x.userData.split("\t")]).collect()

In [ ]:
import geopandas as gpd
l_gdf = gpd.GeoDataFrame(
    Lotes.rawSpatialRDD.map(lambda x: [x.geom]).sample(False, 0.0009,6).collect(),
    columns=["geom"],
    geometry="geom",
    crs = 'epsg:4326'
)

In [ ]:
from sedona.core.spatialOperator import JoinQuery
# haciendo las particiones espaciales
Lotes.analyze()
Lotes.spatialPartitioning(GridType.KDBTREE, 4)
Isocrona.spatialPartitioning(Lotes.getPartitioner())

In [ ]:
from sedona.core.enums import GridType
from sedona.core.enums import IndexType
from sedona.core.spatialOperator import JoinQuery


build_on_spatial_partitioned_rdd = True ## Set to TRUE only if run join query
using_index = True
Isocrona.buildIndex(IndexType.QUADTREE, build_on_spatial_partitioned_rdd)
# union y selección de las entidades espaciales dentro de la isocrona
result = JoinQuery.SpatialJoinQueryFlat(Lotes, Isocrona, using_index, True)

In [ ]:
valores = gpd.GeoDataFrame(
    result.map(lambda x: [x[1].geom, *x[0].userData.split("\t"), *x[1].userData.split("\t")]).collect(),columns=["geom", 'group_inde','Tiempo','layer','Linea','LOTSECT_ID','LOTMANZ_ID','LOTLOTE_ID','LOTZHF_ID','LOTZHG_ID','LOTUNIDAPH','LOTDISTRIT','LOTLSIMBOL','ESTADO_REG','FECHA_REGI','FECHA_DESD','FECHA_HAST','LOTLNUMERO','FRENTE','FONDO'],
    geometry="geom",
    crs= "epsg:4326"
)
#.sample(False, 0.0009,6)

In [ ]:
import numpy as np
valores['Linea'].value_counts()

In [ ]:
valores[['geom','Tiempo']].sample(2000).explore('Tiempo')

In [ ]:
valores.tail(4)

# CRUCE CON USOS E INMUEBLES.

In [ ]:
import os
startpath = '/dbfs/FileStore/tables/Catastro/'

# crea listados por grupo de archivos para cada año junto a un diccionario de la información disponible de catastro
# desde el año 2006 hasta 2022
ph = list()
pre = list()
uso = list()
dict_anio = dict()

for root, dirs, files in os.walk(startpath):
    periodos = [str(a) for a in range(2006,2023)]
    # recorre el arreglo de años   
    if os.path.basename(root) in periodos:
        carpeta = os.path.basename(root)
        ph_predios = '{}/ph_{}.csv'.format(root, carpeta)
        predios = '{}/predios_{}.csv'.format(root, carpeta)
        usos = '{}/usos_{}.csv'.format(root, carpeta)
        lista_interna = list()
        for f in files:
            if 'ph_' in f:
                lista_interna.append(ph_predios)
                ph.append(ph_predios)
            elif 'predios_'in f:
                lista_interna.append(predios)
                pre.append(predios)
            elif 'usos_'in f:
                lista_interna.append(usos)
                uso.append(usos)
            else:
                pass
        dict_anio[carpeta] = lista_interna
        del lista_interna
    else:
        pass

In [ ]:
# se definen los usos definidos y agrupados por los profesionales del equipo OOVS
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Iniciar una sesión de Spark
spark = SparkSession.builder.appName("usos").getOrCreate()

hUsos={
    'Asociar al uso principal':['AREA DE MEZANINE EN PH','DEPOSITO (LOCKERS) PH','DEPOSITO ALMACENAMIENTO PH','PARQUEO CUBIERTO NPH','PARQUEO CUBIERTO PH','PARQUEO LIBRE PH','PISCINAS EN NPH','PISCINAS EN PH','SECADEROS'],
    'Comercio y servicios':['BODEGA COMERCIAL NPH','BODEGA COMERCIAL PH','BODEGA ECONOMICA','BODEGA ECONOMICA(SERVITECA,ESTA.SERVIC.)','BODEGAS DE ALMACENAMIENTO NPH','BODEGAS DE ALMACENAMIENTO PH','CENTRO COMERCIAL GRANDE NPH','CENTRO COMERCIAL GRANDE PH','CENTRO COMERCIAL MEDIANO NPH','CENTRO COMERCIAL MEDIANO PH','CENTRO COMERCIAL PEQUENO NPH','CENTRO COMERCIAL PEQUENO PH','CLUBES PEQUENOS','COMERCIO PUNTUAL NPH O HASTA 3 UNID PH','COMERCIO PUNTUAL PH','CORREDOR COMERCIAL NPH O HASTA 3 UNID PH','CORREDOR COMERCIAL PH','DEPOSITOS DE ALMACENAMIENTO NPH','EDIFICIOS DE PARQUEO NPH','EDIFICIOS DE PARQUEO PH','HOTELES NPH','HOTELES PH','MOTELES, AMOBLADOS, RESIDENCIAS NPH','MOTELES, AMOBLADOS, RESIDENCIAS PH','OFICINA BODEGA Y/O INDUSTRIA PH','OFICINAS EN BODEGAS Y/O INDUSTRIAS','OFICINAS OPERATIVAS','OFICINAS OPERATIVAS(ESTACIONES SERVICIO)','OFICINAS Y CONSULTORIOS NPH','OFICINAS Y CONSULTORIOS PH','PARQUES DE DIVERSION','RESTAURANTES NPH','RESTAURANTES PH','TEATROS Y CINEMAS NPH','TEATROS Y CINEMAS PH', 'PARQUES DE DIVERSION EN P.H.'],
    'Dotacional':['AULAS DE CLASE','CEMENTERIOS','CENTROS MEDICOS EN PH','CLINICAS, HOSPITALES, CENTROS MEDICOS','CLUBES MAYOR EXTENSION','COLEGIOS EN PH','COLEGIOS Y UNIVERSIDADES 1 A 3 PISOS','COLEGIOS Y UNIVERSIDADES 4 O MAS PISOS','COLISEOS','CULTO RELIGIOS EN NPH','CULTO RELIGIOSO EN PH','IGLESIA PH','IGLESIAS','INSTALACIONES MILITARES','INSTITUCIONAL PH','INSTITUCIONAL PUNTUAL','MUSEOS','OFICINAS Y CONSULTORIOS (OFICIAL) NPH','OFICINAS Y CONSULTORIOS (OFICIAL) PH','PLAZAS DE MERCADO'],
    'Industrial':['INDUSTRIA ARTESANAL','INDUSTRIA GRANDE','INDUSTRIA GRANDE PH','INDUSTRIA MEDIANA','INDUSTRIA MEDIANA PH'],
    'Otro':['COCHERAS, MARRANERAS, PORQUERIZAS','ENRAMADAS, COBERTIZOS, CANEYES','ESTABLOS, PESEBRERAS','GALPONES, GALLINEROS','KIOSKOS','LOTE EN PROPIEDAD HORIZONTAL','PISTA AEROPUERTO','SILOS'],
    'Residencial':['HABITACIONAL EN PROPIEDAD HORIZONTAL','HABITACIONAL MAYOR O IGUAL A 4 PISOS NPH O 3 PISOS PH','HABITACIONAL MENOR O IGUAL A 3 PISOS NPH','HABITACIONAL MENOR O IGUAL A 3 PISOS PH','MAYOR O IGUAL A 4 PISOS NPH O 3 PISOS PH']
 }

def homoUso(uso_catastro:str,usos_agregados=hUsos):
    '''
    Toma el uso de la base de predios de catastro y lo agrega a 
    categorias mas sencillas de entender para el análisis de datos del OOVS

    Args:
        input_str (str): Cadena de texto que trae el uso para gregarlo.

    Returns:
        str: retorna el uso agragado y homologado por el equipo del observatorio
    '''
    for k,v in usos_agregados.items():
        if uso_catastro in v:
            return k
        elif uso_catastro not in v:
            pass

# Registrar la función Python como UDF de Spark
homoUso_udf = udf(homoUso, StringType())

In [ ]:
# recorre el arreglo de los archivos de uso y crea un nuevo campo con el uso agregado, luego lo guarda en un archivo nuevo.
for u in uso:
    path = u.replace("/dbfs","dbfs:")
    uso_pdf = spark.read.format("csv").option("header", "true").load(path)
    df = uso_pdf.withColumn("UsoAgregado", homoUso_udf(uso_pdf["DESCRIPCION_USO"]))
    new_name = path.replace('.csv','_agregado.csv') #uso homologado
    df.write.csv(new_name, header=True, mode="overwrite")

In [3]:
[a.path for a in dbutils.fs.ls("/FileStore/tables/Catastro/2010/")]

['dbfs:/FileStore/tables/Catastro/2010/ph_2010.csv',
 'dbfs:/FileStore/tables/Catastro/2010/predios_2010.csv',
 'dbfs:/FileStore/tables/Catastro/2010/usos_2010.csv',
 'dbfs:/FileStore/tables/Catastro/2010/usos_2010_agregado.csv/']

# Conteo usos por lote

In [4]:
from pyspark.sql.functions import sum
uA = spark.read.format("csv").option("header", "true").load('dbfs:/FileStore/tables/Catastro/2010/usos_2010_agregado.csv/')

In [ ]:
#uA.groupBy("LOTLOTE_ID").count().show()

conteoUsos = uA.groupBy("LOTLOTE_ID").pivot("UsoAgregado").count()
#uA.select('DESCRIPCION_USO').where(uA.UsoAgregado.isNull()).groupBy("DESCRIPCION_USO").count().display()

In [ ]:
conteoUsos.display()

In [ ]:
pred_example = spark.read.format("csv").option("header", "true").load('dbfs:/FileStore/tables/Catastro/2010/predios_2010.csv/')

In [ ]:
#pred_example.groupBy("LOTLOTE_ID").count().display()
agrupacion_predios = pred_example.groupBy("LOTLOTE_ID").agg({"AREA_TERRENO":"sum", "AREA_CONSTRUIDA":"sum", "VALOR_AVALUO":"sum", "CHIP":"count"})

agrupacion_predios = agrupacion_predios.withColumnRenamed("count(CHIP)","NumPredios") \
    .withColumnRenamed("sum(VALOR_AVALUO)","ValorAvaluo") \
    .withColumnRenamed("sum(AREA_CONSTRUIDA)","AC") \
    .withColumnRenamed("sum(AREA_TERRENO)","AT")

union = agrupacion_predios.join(conteoUsos, 'LOTLOTE_ID', "inner")

In [ ]:
union.display()

In [ ]:
ph_proce = spark.read.format("csv").option("header", "true").load('dbfs:/FileStore/tables/Catastro/2010/ph_2010.csv')
ph_proce = ph_proce.select("LOTLOTE_ID","NUMERO_UNIDADES_PH")
ph_proce = ph_proce.withColumnRenamed("NUMERO_UNIDADES_PH","Unidades_En_PH")

union2 = union.join(ph_proce, 'LOTLOTE_ID')
# este dato no tiene sentido cruzar solo los predios o lotes que tienen PH

In [ ]:
union2.display()